<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/real_summary3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Beginners Guide to Text Generation using LSTMs**

https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

참조 : https://github.com/williamSYSU/TextGAN-PyTorch


In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 65 kB 632 kB/s eta 0:00:011
  Created wheel for sentence-transformers: filename=sentence_transformers-0.4.0-py3-none-any.whl size=102651 sha256=2f98a74d60169ba6644bb792a3462524331c80bd71667fe77418308e0f74d2b1
  Stored in directory: /root/.cache/pip/wheels/2a/8d/08/4a10448945e7a511061172df2c2a890e0d359e9a2060d89c44
Successfully built sentence-transformers
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

* Text dataset을 만들기 위해 위키백과에서 스크랩핑...

# 여기는 생략 가능

In [ ]:
!pip install wikipedia
import wikipedia as wiki
wiki.set_lang('ko')

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11685 sha256=c30ded6550c5864ef8ffdae7c93296e8fb09e0b76fc1ad2a2665284789e9e48f
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [ ]:
def __search_from_wiki(question,max_rank):
  results = wiki.search(question,results=max_rank)
  print(results)
  contents = []
  for result in results:
    try:
      page = wiki.page(result)
      #print(f"Top wiki result: {page}")
      text = page.content
      ln = len(text)
      print(ln)
      if ln < 4000:
        contents.append(text)
      else:
        contents.append(text[0:4000])
    except Exception as ex:
      print(ex)
  return contents


all_headlines = __search_from_wiki("전래동화", 10)

len(all_headlines)

['동화', '신 전래동화', '꾸러기 수비대', '호시조라 미유키', '아동 문학', '한국의 사찰', '해와 달이 된 오누이', '잠자는 숲속의 미녀', '거북', '이한갈']
685
156
1554
1601
678
885
392
2037
1377
776


10

In [ ]:
all_headlines[3]

'호시조라 미유키(星空 (ほしぞら) みゆき 한국명:김다솜)는 토에이 애니메이션 제작의 애니메이션《스마일 프리큐어!》에 등장하는 가공의 인물이다. 애니메이션에서 목소리를 연기한 성우는 일본판은 후쿠엔 미사토, 한국판은 양정화이다.\n\n\n== 개요 ==\n프리큐어 시리즈 9번째 작품에 등장하는 7대째 프리큐어 전사 큐어 해피로 변신하는 인물. 가족은 부모님이 계시며, 외동딸이다. 머리 모양은 양갈래를 코로네와 소라빵모양처럼 작게 돌돌 말아 노란색 리본으로 묶고 있다.\n그림책과 동화책 읽는 것을 가장 좋아하며, 모든지 \'해피\'한게 좋다는 항상 긍정적이고 해맑은 여중생이다. 첫사랑 상대는 피터팬과 백마 탄 왕자이며, 가장 좋아하는 책은 안데르센의 동화 신데렐라. 기분이 좋아 흥분하면 "울트라 해피!(일본어: ウルトラハッピー!)"라고 말하는 말버릇이 있다. 반면 조금 맘에 안들거나 할때는 입을 삐죽 내밀며 "합뿌뿌~(일본어: ハップップー)"라고 하기도 한다.\n공부도 운동도 별로 못하고 덜렁이에 실수가 많지만, 결코 실망하는 법이 없이 한결같이 앞으로 전진하는 명랑하고 발랄한 성격인 소녀. 겉보기에는 미덥지 못하지만, 항상 모두를 행복하게 하기 위해 최선을 다하며 그 미소를 빼앗으려는 악당들에게도 과감히 맞서는 강하고 상냥한 마음은 친구들과 주변 사람들을 매료시키고있다. 한편 의외로 맘이 여리고 섬세해서 상처받기 쉬운 면도 있지만 눈물 보이는 것을 싫어하며 속으로 꾹꾹 누르며 항상 웃는 얼굴을 보여준다.\n어릴 적에 순수하고 밝은 성격과 달리 친구만들기가 어려운 부끄럼많고 내성적인 성격이였으나, 예전에 할머니집에서 살았던 마을에서 만난 숲의 소녀 덕분에 자신감과 용기를 얻는 계기가 되었다.\n다른 바보 3명과 달리 모든 과목을 못하는 것 같다. 16화에서는 다 아는 속담을 전래동화 드립을 쳤다.\n\n\n== 큐어 해피 ==\n성스러운 빛의 힘을 가진 프리큐어로서 변신한 모습. 이미지 색상은 분홍색.\n\n변신 주문은 "프리큐어 스마일 차지!"\n변신 후 대사

In [ ]:
def clean_text(txt):
    txt = txt.replace('\n','')
    return txt 

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['동화(童話)란 어린이를 독자층으로 하는 이야기를 말한다. 아동 문학의 한 장르이다.판타지나 우화, 프로파간다와는 구분할 필요가 있다.== 나라별 동화 ==그리스의 이솝우화, 이탈리아의 피노키오, 독일의 헨젤과 그레텔 등 동화로 알려진 문학이 있다.=== 한국 ===한국에서는 흔히 어린이를 위한 옛날 이야기를 동화라고 이해하고 있다. 이와 같은 해석은 이 문학 유형이 갖고 있는 고유한 특성을 간접적으로 설명한다고 볼 수 있다. 왜냐하면 전통적 의미에서 동화는 한 지방이나 국가에서 세대를 거치면서 자기 나라말로 구전된 것에서 그 뿌리를 두고 있기 때문이다. 이러한 전통적 동화는 종류가 대단히 한정될 수밖에 없다. 현대에 들어서 동화의 창작은 간과할 수 없는 문학 작업으로 평가되고 있다.한국의 본격적인 창작동화는 1923년 마해송(馬海松)의 〈바위나리와 아기별〉이 발표되고, 이어서 《어린이》·《아이생활》·《별나라》 등 여러 잡지와 구연회(口演會)를 통하여 방정환·고한승(高漢承)·진장섭(秦長燮)·정인섭·이정호(李定鎬) 등 색동회 동인들과 기타 이주홍(李周洪) 등의 작가들이 창작활동을 활발히 전개함으로써 비로소 기반을 쌓게 되었다. 현재도 보리출판사, 사계절출판사, 예영커뮤니케이션 등 다수의 출판사에서 어린이들을 위한 창작동화들을 출판하고 있다.== 나라별 동화 작가 ==== 각주 ==',
 '《신 전래동화》는 2018년에 개봉한 대한민국의 영화이다.== 캐스팅 ==== 같이 보기 ==2018년 대한민국의 영화 목록== 외부 링크 ==신 전래동화 - 네이버 영화신 전래동화 - 다음 영화신 전래동화 - 무비스트신 전래동화 - 한국영화 데이터베이스',
 '《꾸러기 수비대》 (혹은《출동! 12레인저》)는 십이지를 소재로 한 애니메이션이다.일본에서는 《십이전지 폭렬 에토레인저》(十二戦支 爆烈エトレンジャー)라는 이름으로 NHK 위성 제2텔레비전을 통해 1995년에 방영되었다.== 개요 ==한국어 더빙판으로는 1996년 KBS 2TV에서 방영한 《꾸러기 수비대》와 투니버스에서

In [ ]:

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[19, 234],
 [19, 234, 235],
 [19, 234, 235, 34],
 [19, 234, 235, 34, 236],
 [19, 234, 235, 34, 236, 237],
 [19, 234, 235, 34, 236, 237, 50],
 [19, 234, 235, 34, 236, 237, 50, 238],
 [19, 234, 235, 34, 236, 237, 50, 238, 239],
 [19, 234, 235, 34, 236, 237, 50, 238, 239, 84],
 [19, 234, 235, 34, 236, 237, 50, 238, 239, 84, 10]]

In [ ]:
len(inp_sequences)

2070

In [ ]:
total_words

1606

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [ ]:
predictors.shape

(2070, 430)

In [ ]:
label.shape

(2070, 1606)

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 430, 10)           16060     
_________________________________________________________________
lstm (LSTM)                  (None, 100)               44400     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1606)              162206    
Total params: 222,666
Trainable params: 222,666
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(predictors, label, epochs=10, verbose=5)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
print (generate_text("b", 20, model, max_sequence_len))


B The The The The The The The The The The The The The The The The The The The The


In [ ]:
print (generate_text("이탈리아", 30, model, max_sequence_len))

이탈리아 The The The The The The The The The The The The The The The The The The The The The Sbs Sbs Sbs Sbs Sbs Sbs Sbs Sbs Sbs


# 여기서부터 본론...

In [ ]:
from sentence_transformers import SentenceTransformer
# embedder download...
embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

100%|██████████| 1.80G/1.80G [00:46<00:00, 38.8MB/s]


In [ ]:
document = """
주호영 국민의힘 원내대표는 22일 고위공직자범죄수사처(공수처)법 개정과 가덕도 신공항 건설 등을 밀어붙이고 있는 문재인 정권과 더불어민주당을 향해 "이제 끝이 보인다"며 "짓밟힌 풀들이 아우성 치는 국민적 저항에 직면할 것"이라고 경고했다.
주 원내대표는 이날 자신의 페이스북에 "문재인 정권이 공수처법 개정을 위한 '군사작전'에 돌입하겠다고 엄포를 놓고 있다"며 "정의당을 끌어들이기 위해 꼼수 선거법에 묶어 '패스트트랙'이라는 불법·탈법으로 만들어낸 공수처법을 시행도 해보지 않고 고치려 하는 것"이라고 지적했다.
이어 주 원내대표는 "야당 원내대표인 제게 문재인 대통령은 사람 좋아보이는 표정으로 '공수처는 야당의 동의 없이는 절대 출범할 수 없는 것'이라고 얘기했고, 야당이 유엔 안보리 상임이사국처럼 공수처장 임명에 '비토권'을 행사할 수 있는데 무얼 걱정하느냐고, 여당 사람들이 우리를 속였다"며 "거짓말이라는 비난을 개의치 않는 사람들"이라고 꼬집었다.
주 원내대표는 "이해찬 전 민주당 대표가 얘기한 '민주당 20년 집권'의 토대가 올해 안에 완성된다"며 "탈원전과 동남권 신공항은 문 대통령이 대선 공약으로 내건 사업이니 여기에 불법이 있었다고 시비를 거는 것은 민주주의를 부정하는 것이라고 청와대 출신 윤건영 민주당 의원이 윽박지른다. 이제 '민주주의 없는 민주당'이 법위에 군림하는 '반민주'를 거리낌없이 획책하는 것"이라고 언급했다.
그러면서 주 원내대표는 "표를 얻기 위해 나라 곳간을 다 허물어뜨렸고, 재정 운용에서 신중함은 사라졌다"며 "괴물 공수처가 출범하면 공무원 누구나 대통령과 권력이 지시하는 범죄행위에 거리낌 없이 가담할 것이다. 청와대와 권부 요직에 앉아 불법으로 각종 이권을 챙기는 권력자들에 대한 사건이 불거져도 공수처가 사건을 가져가 버리면 그만"이라고 우려했다.
주 원내대표는 "문 대통령은 제게 '공수처는 고위 공직자들을 처벌하는 것인데 왜 야당이 반대하는지 이해할 수 없다'고 했는데, 그런 분이 청와대와 대통령 주변을 감시하는 특별감찰관은 취임 이후 지금까지 왜 임명하지 않았는가"라며 "공수처는 권력형 비리의 쓰레기 하치장, 종말 처리장이 될 것"이라고 비판했다.
문재인 정부를 향해 주 원내대표는 "문 대통령과 그 사도들은 법치가 미치지 않는 무오류의 화신이 될 것"이라며 "오류를 인정하지 않는 존재가 바로 신이며 그 아래에는 자신들의 지도자를 목숨바쳐 지킴으로서 정의를 실현하겠다는 추종자들로 넘쳐 난다. 공수처는 지도자의 신성을 인정하지 않는 세력을 정죄하는 수단으로 전락할 것"이라고 질타했다.
주 원내대표는 "저도 법조인이지만 대통령과 공수처장이 마음대로 검사들과 수사관들을 임명하는 이 끔찍한 사법기구가 어떤 일을 할지 두렵기만 하다"며 "공수처는 검찰과 경찰 위에 있는 사법기구로, 헌법과 법으로 독립성을 보장하는 검찰총장을 이렇게 핍박하는 정권이 공수처를 어떻게 운영할지 불을 보듯 뻔한 일"이라고 예측했다.
그러면서 주 원내대표는 "추미애 법무장관을 앞장 세워 윤석열 검찰의 권력 비리 수사를 저지하려다가 난관에 봉착하자 무슨 수를 써서라도 공수처를 출범시키려 한다. 공수처장 자리에는 추미애보다 더 한 막무가내 내 편을 앉힐 게 분명한 것"이라며 "문 정권의 파렴치와 오만함을 최전선에서 온 몸으로 겪어온 저로서는 민주당이 내일부터 국회에서 보일 행태가 환히 보인다. 180석의 민주당이 또 군사작전을 개시하면 그걸 누가 막겠는가"라고 성토했다.
주 원내대표는 "공수처법을 막을 힘이 우리 야당에게는 없다. 삭발하고 장외투쟁해 봐야 눈 하나 깜짝할 사람들이 아닌 것"이라며 "대란대치(大亂大治), 세상을 온통 혼돈 속으로 밀어넣고 그걸 권력 유지에 이용한다는 게 이 정권의 통치기술"이라고 규탄했다.
아울러 주 원내대표는 "권력은 바람, 국민은 풀이다. 바람이 불면 청보리 밭의 보리가 눕는다"며 "권력은 풀들이 다시는 일어서지 못하도록 풀을 짓밟지만 풀들은 다시 일어난다. 시인 김수영은 '바람보다 먼저 눕지만, 바람보다 먼저 일어나는' 민초의 힘을 노래했다"고 말했다.
마지막으로 주 원내대표는 "문재인 정권은 이제 곧 국회에서 광장에서 짓밟힌 풀들이 일어서서 아우성치는 모습을 지켜보게 될 것"이라며 "대란대치를 끝장내려는 국민적 저항에 직면할 것"이라고 거듭 강조했다.
"""

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([document])
total_words = len(tokenizer.word_index) + 1

In [ ]:
print(total_words)

395


In [ ]:
word_set = []
for word,index in tokenizer.word_index.items():
    word_set.append(word)
    
word_set

['원내대표는',
 '주',
 '것',
 '이라고',
 '며',
 '문재인',
 '않는',
 '문',
 '이라며',
 '이제',
 '풀들이',
 '수',
 '대통령과',
 '공수처는',
 '될',
 '있는',
 '향해',
 '보인다',
 '짓밟힌',
 '국민적',
 '저항에',
 '직면할',
 '정권이',
 '위해',
 '공수처법을',
 '제게',
 '대통령은',
 "'공수처는",
 '없는',
 '야당이',
 '공수처장',
 '사람들이',
 '민주당',
 '그러면서',
 '공수처가',
 '청와대와',
 '왜',
 '그',
 '인정하지',
 '이',
 '공수처를',
 '권력',
 '게',
 '정권의',
 '민주당이',
 '국회에서',
 '그걸',
 '권력은',
 '먼저',
 '주호영',
 '국민의힘',
 '22일',
 '고위공직자범죄수사처',
 '공수처',
 '법',
 '개정과',
 '가덕도',
 '신공항',
 '건설',
 '등을',
 '밀어붙이고',
 '정권과',
 '더불어민주당을',
 '끝이',
 '아우성',
 '치는',
 '경고했다',
 '이날',
 '자신의',
 '페이스북에',
 '공수처법',
 '개정을',
 '위한',
 "'군사작전'에",
 '돌입하겠다고',
 '엄포를',
 '놓고',
 '있다',
 '정의당을',
 '끌어들이기',
 '꼼수',
 '선거법에',
 '묶어',
 "'패스트트랙'이라는",
 '불법·탈법으로',
 '만들어낸',
 '시행도',
 '해보지',
 '않고',
 '고치려',
 '하는',
 '지적했다',
 '이어',
 '야당',
 '원내대표인',
 '사람',
 '좋아보이는',
 '표정으로',
 '야당의',
 '동의',
 '없이는',
 '절대',
 '출범할',
 "것'이라고",
 '얘기했고',
 '유엔',
 '안보리',
 '상임이사국처럼',
 '임명에',
 "'비토권'을",
 '행사할',
 '있는데',
 '무얼',
 '걱정하느냐고',
 '여당',
 '우리를',
 '속였다',
 '거짓말이라는',
 '비난을',
 '개의치',
 '사람들',
 '꼬

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input,
                                     Dense, 
                                     BatchNormalization, 
                                     LeakyReLU,
                                     Softmax,
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten)
import matplotlib.pyplot as plt

In [ ]:
#w = tf.constant([[1,2,3,4,5,4,3,2,1],[1,2,3,4,5,4,3,2,1],[1,2,3,4,5,4,3,2,1],[1,2,3,4,5,4,3,2,1]])
w = tf.random.normal([1,30,395])
print(w.shape)
def feature_encode(w):
    #embeddings = tf.Variable(tf.zeros([w.shape[0],1024]))
    embeddings = []
    idx = 0
    for z in w:
        generated_text = ""
        for v in z:
            key = np.argmax(v)
            #print(key)
            generated_text += word_set[key] + ' '
        print(generated_text)
        embeddings.append(embedder.encode([generated_text],show_progress_bar=False)[0])
        #embeddings[idx].assign(embedder.encode([generated_text],show_progress_bar=False)[0])
        
    return tf.Variable(embeddings)

e = feature_encode(w)
print(e.shape)


(1, 30, 395)


RuntimeError: CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 15.90 GiB total capacity; 0 bytes already allocated; 923.75 MiB free; 0 bytes reserved in total by PyTorch)

In [ ]:
from keras import backend as K
from keras.layers import Layer

class sentence_encoder(Layer):

    def __init__(self, output_dim,encoder_func, **kwargs):
        self.output_dim = output_dim
        self.word_set = word_set
        self.encoder = encoder_func
        super(sentence_encoder, self).__init__(**kwargs)

    def build(self, input_shape):
        # 이 레이어에 대해 학습가능한 가중치 변수를 만듭니다.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(sentence_encoder, self).build(input_shape)  # 끝에서 꼭 이 함수를 호출하십시오

    def call(self, x):
        return self.encoder(x)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [ ]:
e = sentence_encoder(1024,feature_encode)(w)
e

tf.Tensor(
[[[-0.58914864 -1.3004026  -1.5005717  ...  0.5178405  -0.56622934
   -0.43043807]
  [-0.33853483  2.070352   -0.9864853  ... -0.2612557   1.5971978
   -1.200729  ]
  [-0.90393573  0.88843054  0.35754013 ... -0.6996179  -0.51393735
    0.33115798]
  ...
  [-0.15646507 -0.52768546  0.36976302 ... -1.0549747  -1.858456
   -2.326697  ]
  [-1.1181496   0.832948   -0.15987489 ...  0.8270664   0.8661089
   -0.34101778]
  [-0.8063847  -0.20269231 -0.26659986 ...  0.82387286  0.93095785
    2.5183713 ]]], shape=(1, 30, 395), dtype=float32)


<tf.Tensor: shape=(1, 1024), dtype=float32, numpy=
array([[ 0.43130967, -0.11070471,  0.5162863 , ..., -0.28348795,
         0.5051057 ,  0.10074985]], dtype=float32)>

In [ ]:

def make_generator_model(max_length,total_words):
    input = Input(shape=(1000,), dtype='int32') 
    x1 = Dense(1024, use_bias=False)(input)
    x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    #x1 = Dense(max_length*total_words*2, use_bias=False)(x1)
    #x1 = BatchNormalization()(x1)
    #x1 = LeakyReLU()(x1)
    
    x1 = Dense(max_length*total_words, use_bias=False, activation='tanh')(x1)    
    x1 = Reshape((max_length, total_words))(x1)   
    
    x2 = sentence_encoder(1024,feature_encode)(x1)
    

    model = Model(input,[x1,x2])
    
    '''
    model = tf.keras.Sequential()
    model.add(Dense(256, use_bias=False, input_shape=(1000,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    assert model.output_shape == (None,256) 

    model.add(Dense(256*2, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    assert model.output_shape == (None,256*2) 
    
    model.add(Dense(total_words, use_bias=False))
    model.add(BatchNormalization())
    model.add(Softmax())

    assert model.output_shape == (None,total_words)
    '''
    model.summary()
    return model

generator = make_generator_model(30,total_words)

Tensor("reshape_3/Reshape:0", shape=(None, 30, 395), dtype=float32)


NotImplementedError: in user code:

    <ipython-input-50-827381adfd1f>:21 call  *
        return self.encoder(x)
    <ipython-input-53-dc6dc87b3a45>:10 feature_encode  *
        key = np.argmax(v)
    <__array_function__ internals>:6 argmax  **
        
    /opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:1153 argmax
        return _wrapfunc(a, 'argmax', axis=axis, out=out)
    /opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:58 _wrapfunc
        return _wrapit(obj, method, *args, **kwds)
    /opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:47 _wrapit
        result = getattr(asarray(obj), method)(*args, **kwds)
    /opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:85 asarray
        return array(a, dtype, copy=False, order=order)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/ops.py:848 __array__
        " a NumPy call, which is not supported".format(self.name))

    NotImplementedError: Cannot conver

In [ ]:
# Create a random noise and generate a sample
noise = tf.random.normal([10, 1000])
generated_code = generator(noise, training=False)

generated_text = ""

for w in generated_code[0]:
    key =int(tf.math.argmax(w))+1
    generated_text += word_set[key] + ' '
    
generated_text    

"불거져도 표를 이어 국민의힘 통치기술 한 동의 저지하려다가 등을 바람보다 전 내건 어떤 이렇게 선거법에 아닌 권부 다시 것이다 선거법에 검찰과 동남권 몸으로 세력을 '반민주'를 저지하려다가 불을 저항에 20년 않고 "

In [ ]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(Dense(256, use_bias=False, input_shape=(1024,)))
    model.add(BatchNormalization())
    model.add(LeakyReLU())
    #model.add(Reshape((256,)))
    assert model.output_shape == (None,256) # Note: None is the batch size

    model.add(Dense(64, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    #model.add(Reshape((64,)))
    assert model.output_shape == (None,64) # Note: None is the batch size

    model.add(Dense(32, use_bias=False))
    model.add(BatchNormalization())
    model.add(LeakyReLU())    
    #model.add(Reshape((64,)))
    assert model.output_shape == (None,32) # Note: None is the batch size

    model.add(Dense(1))
    #model.add(Softmax())    
    model.summary()
    return model

discriminator = make_discriminator_model()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               262144    
_________________________________________________________________
batch_normalization_3 (Batch (None, 256)               1024      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 64)                16384     
_________________________________________________________________
batch_normalization_4 (Batch (None, 64)                256       
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 64)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)               

In [ ]:
#generated_encode = embedder.encode([generated_sum])
decision = discriminator(generated_embedding)
print (decision)

In [ ]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
BATCH_SIZE = 32
EPOCHS = 60
noise_dim = 50
seed = tf.random.normal([1, noise_dim])

In [ ]:
def generated_code_to_text(generated_code,total_words):
  generated_sum = []
  generated_code = [c * total_words for c in generated_code]
  for g in generated_code:
    summary_txt = ""
    for idx in g:
      #print(int(idx))
      for word,index in tokenizer.word_index.items():
          if index == int(idx):
              summary_txt += word + ' '
              break
    generated_sum.append(summary_txt)
  return generated_sum, embedder.encode(generated_sum)

In [ ]:
noise = tf.random.normal([BATCH_SIZE, 50])
generated_code = generator(noise, training=False)
generated_text,generated_embedding = generated_code_to_text(generated_code,total_words)
print(generated_text[0])

In [ ]:
@tf.function
def train_step(images):
  
    # 1 - Create a random noise to feed it into the model
    # for the text generation
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    
    # 2 - Generate text and calculate loss values
    # GradientTape method records operations for automatic differentiation.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_code = generator(noise, training=True)
      generated_text,generated_embedding = generated_code_to_text(generated_code,total_words)
      real_embedding = embedder.encode(images)
      real_output = discriminator(real_embedding, training=True)
      fake_output = discriminator(generated_embedding, training=True)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    # 3 - Calculate gradients using loss values and model variables
    # "gradient" method computes the gradient using 
    # operations recorded in context of this tape (gen_tape and disc_tape).
    
    # It accepts a target (e.g., gen_loss) variable and 
    # a source variable (e.g.,generator.trainable_variables)
    # target --> a list or nested structure of Tensors or Variables to be differentiated.
    # source --> a list or nested structure of Tensors or Variables.
    # target will be differentiated against elements in sources.

    # "gradient" method returns a list or nested structure of Tensors  
    # (or IndexedSlices, or None), one for each element in sources. 
    # Returned structure is the same as the structure of sources.
    gradients_of_generator = gen_tape.gradient(gen_loss, 
                                               generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                discriminator.trainable_variables)
    
    # 4 - Process  Gradients and Run the Optimizer
    # "apply_gradients" method processes aggregated gradients. 
    # ex: optimizer.apply_gradients(zip(grads, vars))
    """
    Example use of apply_gradients:
    grads = tape.gradient(loss, vars)
    grads = tf.distribute.get_replica_context().all_reduce('sum', grads)
    # Processing aggregated gradients.
    optimizer.apply_gradients(zip(grads, vars), experimental_aggregate_gradients=False)
    """
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [ ]:
dataset = []
for i in range(1000):
  dataset.append(document)

In [ ]:
import time
from IPython import display # A command shell for interactive computing in Python.

def train(dataset, epochs):
  # A. For each epoch, do the following:
  for epoch in range(epochs):
    start = time.time()
    # 1 - For each batch of the epoch, 
    for image_batch in dataset:
      # 1.a - run the custom "train_step" function
      # we just declared above
      train_step(image_batch)

    # 4 - Print out the completed epoch no. and the time spent
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [ ]:
train(dataset, EPOCHS)